In [ ]:
!git clone https://github.com/pytorch/fairseq


Cloning into 'fairseq'...
remote: Enumerating objects: 34477, done.
remote: Counting objects: 100% (86/86), done.
remote: Compressing objects: 100% (76/76), done.
remote: Total 34477 (delta 22), reused 39 (delta 8), pack-reused 34391
Receiving objects: 100% (34477/34477), 24.03 MiB | 26.49 MiB/s, done.
Resolving deltas: 100% (25034/25034), done.


In [ ]:
cd fairseq


/content/fairseq


In [ ]:
!pip install --editable ./

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/fairseq
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 KB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 KB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.5/269.5 KB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 KB 16.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.whl size=141231 sha256=080ef3fcee72c6af066f778f955828cd7996abce0309675120cafdda65f2bd75
  Stored in directory: /root/.cache/pip/wheels/c8/d0/ab/d43c02eaddc5b9004db86950802442ad9a26f279c619e28da0
Successfully built

In [ ]:
!git clone https://github.com/alirezamshi/small100

Cloning into 'small100'...
remote: Enumerating objects: 48, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 48 (delta 19), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (48/48), 87.54 KiB | 3.65 MiB/s, done.


In [ ]:
!git clone https://github.com/pytorch/fairseq -q
%cd fairseq
!pip uninstall numpy -q -y
!pip install wandb -q
!pip install --editable ./ -q
%cd ..

import torch

torch.cuda.get_device_name(0)

!wget -qq "https://dl.fbaipublicfiles.com/m2m_100/spm.128k.model"
!wget -qq "https://dl.fbaipublicfiles.com/m2m_100/data_dict.128k.txt"
!wget -qq "https://dl.fbaipublicfiles.com/m2m_100/model_dict.128k.txt"
!wget -qq "https://dl.fbaipublicfiles.com/m2m_100/language_pairs_small_models.txt"
!wget "https://dl.fbaipublicfiles.com/m2m_100/418M_last_checkpoint.pt"

! pip install sentencepiece -q


import pandas as pd
import torch
import numpy as np
import os
import random

def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.enabled = False
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

set_seed(7)



# loading the data
df = pd.read_csv('Local_35k.csv')
t1 = df[:32000]

t2 = df[32000:]

t1.to_csv('train.csv',index = None)
t2.to_csv('test.csv' , index = None)


PATH_TO_DATASET = "./"  #Where you stored the dataset

train = pd.read_csv(os.path.join(PATH_TO_DATASET, "train.csv"))

#Remove any possible duplicates
train = train.drop_duplicates(subset=["Arabic", "Swahili"])

#Lowercase and remove trailing spaces
train["Arabic"] = train.apply(lambda x: (x.Arabic).strip().lower(), axis=1)
train["Swahili"] = train.Swahili.apply(lambda x: x.lower())

train = train[["Arabic", "Swahili"]]
train.columns = ["input_text", "target_text"]

#Train 95% / Validation 5% Split
validation = train.sample(frac=0.05).astype(str)
train = train.drop(index=validation.index).astype(str)

train_txt = "\n".join(train.input_text.values.tolist())

file = open("Arabic_txt_train.txt", "w")
file.write(train_txt)
file.close()


train_target_txt = "\n".join(train.target_text.values.tolist())

file = open("Swahili_txt_train.txt", "w")
file.write(train_target_txt)
file.close()

validation_txt = "\n".join(validation.input_text.values.tolist())

file = open("Arabic_txt_validation.txt", "w")
file.write(validation_txt)
file.close()


validation_target_txt = "\n".join(validation.target_text.values.tolist())

file = open("Swahili_txt_validation.txt", "w")
file.write(validation_target_txt)
file.close()


!python fairseq/scripts/spm_encode.py \
        --model spm.128k.model \
        --output_format=piece \
        --inputs=Arabic_txt_train.txt \
        --outputs=train.ar

!python fairseq/scripts/spm_encode.py \
        --model spm.128k.model \
        --output_format=piece \
        --inputs=Swahili_txt_train.txt \
        --outputs=train.sw
!python fairseq/scripts/spm_encode.py \
        --model spm.128k.model \
        --output_format=piece \
        --inputs=Arabic_txt_validation.txt \
        --outputs=val.ar

!python fairseq/scripts/spm_encode.py \
        --model spm.128k.model \
        --output_format=piece \
        --inputs=Swahili_txt_validation.txt \
        --outputs=val.sw
!fairseq-preprocess \
    --source-lang sw --target-lang ar \
    --trainpref train \
    --validpref val \
    --thresholdsrc 0 --thresholdtgt 0 \
    --destdir data_bin \
    --srcdict model_dict.128k.txt --tgtdict model_dict.128k.txt
from google.colab import drive
drive.mount('/content/drive')
import wandb

wandb.login()
!fairseq-train data_bin \
  --finetune-from-model  "/content/418M_last_checkpoint.pt"\
  --save-dir '/content/drive/MyDrive/ironside_nmt\checkpointsm2m' \
  --task translation_multi_simple_epoch \
  --encoder-normalize-before \
  --lang-pairs 'ar-sw' \
  --batch-size 10 \
  --decoder-normalize-before \
  --encoder-langtok src \
  --decoder-langtok \
  --criterion cross_entropy \
  --optimizer adafactor \
  --lr-scheduler cosine \
  --lr 3e-05 \
  --max-update 40000 \
  --update-freq 2 \
  --save-interval 1 \
  --save-interval-updates 5000 \
  --keep-interval-updates 10 \
  --no-epoch-checkpoints \
  --log-format simple \
  --log-interval 2 \
  --patience 10 \
  --arch transformer_wmt_en_de_big \
  --encoder-layers 12 --decoder-layers 12 \
  --share-decoder-input-output-embed --share-all-embeddings \
  --ddp-backend no_c10d \
  --max-epoch 10 \
  --wandb-project "Swahili M2M"

  !fairseq-preprocess \
    --source-lang ar --target-lang sw \
    --testpref val \
    --thresholdsrc 0 --thresholdtgt 0 \
    --destdir data_bin \
    --srcdict data_dict.128k.txt --tgtdict data_dict.128k.txt
!fairseq-generate "data_bin/"  --batch-size 32 \
      --path "checkpoint/checkpoint_best.pt" \
      --fixed-dictionary model_dict.128k.txt \
      -s ar -t sw \
      --remove-bpe 'sentencepiece' \
      --beam 5 \
      --task translation_multi_simple_epoch \
      --lang-pairs language_pairs_small_models.txt \
      --decoder-langtok \
      --encoder-langtok src \
      --gen-subset test > gen_out.txt
      # --scoring bleu > "bpe_pcm_en_550_32_300_embed_test_results_l.txt"

In [ ]:
!pip install torch torchvision torchaudio
!pip install transformers datasets

import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset

# Load the datasets
data_ar = load_dataset('text', data_files='Arabic.txt')
data_sw = load_dataset('text', data_files='Awahili.txt')



# Load the tokenizer and model



tokenizer = AutoTokenizer.from_pretrained('Helsinki-NLP/opus-mt-ar-sw')
model = AutoModelForSeq2SeqLM.from_pretrained('Helsinki-NLP/opus-mt-ar-sw')

# Define the training arguments
training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    evaluation_strategy='epoch',
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    learning_rate=5e-5,
    save_total_limit=2,
    save_steps=5000,
    logging_steps=5000,
    warmup_steps=500,
    output_dir='./results',
    overwrite_output_dir=True,
)

# Define the trainer
trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=data_ar['train'],
    eval_dataset=data_sw['train'],
)

# Train the model
trainer.train()


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-26b9f7a203bd38dd/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

FileNotFoundError: ignored